This notebook trains a KNN model on 80% of the training data, finds the optimal value of k, and reports f1 score on the dev data.

In [1]:
import numpy as np
import pandas as pd
import zipfile
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [2]:
# Unzip data files into the "csv" subdirectory

# **IMPORTANT**  This will overwrite existing files in the "csv" folder in your local repo
# with the most recent data files from the data.zip file

unzip_files = zipfile.ZipFile("data.zip", "r")
unzip_files.extractall()
unzip_files.close()

In [3]:
# Load csv files into pandas dataframes
# train_data = pd.read_csv('../W207_CrimeClassification/csv/train_data.csv')
# train_labels = pd.read_csv('../W207_CrimeClassification/csv/train_labels.csv')
# dev_data = pd.read_csv('../W207_CrimeClassification/csv/dev_data.csv')
# dev_labels = pd.read_csv('../W207_CrimeClassification/csv/dev_labels.csv')
# train_data_all = pd.read_csv('../W207_CrimeClassification/csv/train_data_all.csv')
# train_labels_all = pd.read_csv('../W207_CrimeClassification/csv/train_labels_all.csv')
# test_data_all = pd.read_csv('../W207_CrimeClassification/csv/test_data_all.csv')

# NOTE: this didn't work right, maybe because np.savetxt was used to create the csv files in the Data Set up notebook

In [4]:
# Load csv files into numpy arrays
train_data = np.loadtxt('csv/train_data.csv', delimiter=",")
train_labels = np.loadtxt('csv/train_labels.csv', dtype=str, delimiter=",")
dev_data = np.loadtxt('csv/dev_data.csv', delimiter=",")
dev_labels = np.loadtxt('csv/dev_labels.csv', dtype=str, delimiter=",")
train_data_all = np.loadtxt('csv/train_data_all.csv', delimiter=",")
train_labels_all = np.loadtxt('csv/train_labels_all.csv', dtype=str, delimiter=",")
test_data_all = np.loadtxt('csv/test_data_all.csv', delimiter=",")

In [5]:
# print shapes to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)
print("train_data_all shape is", train_data_all.shape)
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

train_data shape is (702439, 6)
train_labels shape is (702439,)
dev_data shape is (175610, 6)
dev_labels shape is (175610,)
train_data_all shape is (878049, 6)
train_data_all shape is (878049, 6)
train_labels_all shape is (878049,)
test_data_all shape is (884262, 6)


In [39]:
# Set up functions for training models and finding optimal value of k and reporting accuracy
def TrainKNN(data, labels, k=5):
    """This function takes as input a set of matching training data and labels,
    test data, and can accept different values of k. It trains a KNN model on the training data
    and returns the model.
    
    This function does not require labels for the test data, so this model can be trained on either
    the full set of training data, with predictions created from the test data from Kaggle OR
    it can be run on our 80% training data with predictions created from the development data."""
    KNN = KNeighborsClassifier(n_neighbors=k, n_jobs = -1)
    KNN.fit(data, labels)
    return KNN

def find_k(data, labels, dev_data, dev_labels, k_values):
    """Find optimal value of k.  Note that this cannot be used on test data from Kaggle 
    because we do not have labels for that data.  This function is intended to only be used
    in the development stage with the development data."""
    for k in k_values:
        
        KNN = TrainKNN(data, labels, k)
        predictions = KNN.predict(dev_data)
        f1 = metrics.f1_score(dev_labels, predictions, average = "weighted")
        headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE","DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING","FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL","OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY","SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE","SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
        pp = KNN.predict_proba(dev_data)
        logloss = metrics.log_loss(dev_labels, pp, labels = headers)
        
        # Print accuracy and F1 score for each value of k
        print("For k =", k, "the F1 score is", round(f1, 4))
        print("For k =", k, "the Log Loss score is", round(logloss, 4))
    print("\n")
    

In [12]:
neigh = KNeighborsClassifier(n_neighbors = 2, n_jobs = -1)
neigh.fit(train_data, train_labels)
pp = neigh.predict_proba(dev_data)

In [32]:
headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE","DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING","FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL","OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY","SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE","SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]

print(metrics.log_loss(dev_labels, pp, labels = headers))

24.5542421915


In [40]:
# Find the optimal value of k using the 80% training data and the development data
k_values = [1, 3, 5, 7, 9, 11, 15]
find_k(train_data, train_labels, dev_data, dev_labels, k_values)

C:\Users\Kippy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


For k = 1 the F1 score is 0.2006
For k = 1 the Log Loss score is 27.6013


C:\Users\Kippy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


For k = 3 the F1 score is 0.1552
For k = 3 the Log Loss score is 22.3636
For k = 5 the F1 score is 0.154
For k = 5 the Log Loss score is 19.0689
For k = 7 the F1 score is 0.1533
For k = 7 the Log Loss score is 16.6886
For k = 9 the F1 score is 0.1526
For k = 9 the Log Loss score is 14.8641
For k = 11 the F1 score is 0.1515
For k = 11 the Log Loss score is 13.3879
For k = 15 the F1 score is 0.1492
For k = 15 the Log Loss score is 11.2378




In [ ]:
# Create zip file of predictions for submission to Kaggle 

# Not written for this model because this was our baseline
# But this is where that zip file would be created in the workflow for other models